## Testes

In [ ]:
# tabela_completa_saida = []

# asfd = pd.read_excel(r'C:\Users\Jose Nelio\Documents\GitHub\ipm_2025\data\efd_base\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811\EFD-Base-Saida_IPM_2027_AnoBase2025_Provisório_parte2_20250811.xlsx',
#                      dtype=str, sheet_name='Dados EFD Saida', skiprows=1, )
# asfd['parte_arquivo'] = arquivo.split('_')[-2]
# asfd['CPF/CNPJ'] = asfd['CPF/CNPJ'].str.zfill(14)



In [ ]:
# tabela_completa = []


# for arquivo in os.listdir(pasta):
#     if arquivo.endswith('.xlsx') and 'Entrada' in arquivo:
#         caminho_arquivo = os.path.join(pasta, arquivo)
#         print (f'Lendo {arquivo}')
#         df = pd.read_excel(caminho_arquivo, sheet_name='Dados EFD Entrada',skiprows=1)
#         df['parte'] = arquivo.split('_')[-2]
#         tabela_completa.append(df)


# df_final = pd.concat(tabela_completa, ignore_index=True)
# tipos = {
#     'Inscrição Estadual': 'string',
#     'CPF/CNPJ': 'string',
#     'Ano de referência': 'string',
#     'Mês de apuração': 'string',
#     'Hash do arquivo transmitido': 'string',
#     'Origem da Informação': 'string',
#     'Cód. Município Origem D100': 'string',
#     'CFOP': 'string',
#     # 'Valor Operação': 'float',
#     # 'Valor BC ICMS': 'float',
#     'Declaração utilizado?': 'string',
#     'Registro utilizado no calculo VA?': 'string'
# }
# df_final = df_final.astype(tipos)


# df_final.to_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\efd_base\\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811\\efd_base_entradas.parquet')


## Codigo

In [ ]:
import pandas as pd
import os
import duckdb
from tqdm import tqdm

## LOCALIZACAO

In [ ]:
# Caminho da pasta onde estão os arquivos
pasta_endereco = r"C:\Users\Jose Nelio\Documents\GitHub\ipm_2025\data\dados_cadastrais\CIPM_DadosCadastrais_IPM_2027_AnoBase2025_Projetado_20250708\DadosCadastrais_IPM_2027_AnoBase2025_Projetado_20250708.xlsx"
df_endereco = pd.read_excel(pasta_endereco, dtype=str ,sheet_name='Dados Cadastrais', skiprows=1, usecols= ('CPF/CNPJ', 'Localidade atual'))
df_endereco.rename(columns={'CPF/CNPJ': 'CPFCNPJ', 'Localidade atual': 'localidade_atual'}, inplace=True)
                            
df_endereco

In [ ]:
df_endereco.to_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\parquet\\endereco.parquet')
df_endereco.info()

## ENTRADAS

In [ ]:
# Caminho da pasta onde estão os arquivos
pasta_entrada = r'C:\Users\Jose Nelio\Documents\GitHub\ipm_2025\data\efd_base\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811'


In [ ]:
tabela_completa_entrada = []

for arquivo in os.listdir(pasta_entrada):
    if arquivo.endswith('.xlsx') and 'Entrada' in arquivo:
        caminho_arquivoe = os.path.join(pasta_entrada,arquivo)
        print(f'Lendo arquivo: {arquivo}')
        df_entrada = pd.read_excel(caminho_arquivoe, sheet_name='Dados EFD Entrada', skiprows=1)
        df_entrada['parte_arquivo'] = arquivo.split('_')[-2]
        tabela_completa_entrada.append(df_entrada)

df_entrada = pd.concat(tabela_completa_entrada, ignore_index=True)
schema_entrada = {
    'Inscrição Estadual': 'string',
    'CPF/CNPJ': 'string',
    'Ano de referência': 'string',
    'Mês de apuração': 'string',
    'Hash do arquivo transmitido': 'string',
    'Origem da Informação': 'string',
    'Cód. Município Origem D100': 'string',
    'CFOP': 'string',
    # 'Valor Operação': 'float',
    # 'Valor BC ICMS': 'float',
    'Declaração utilizado?': 'string',
    'Registro utilizado no calculo VA?': 'string'
}
# Remove pontos e troca vírgula por ponto
df_entrada['Valor Operação'] = df_entrada['Valor Operação'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_entrada['Valor BC ICMS'] = df_entrada['Valor BC ICMS'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)

# Converte para float
df_entrada['Valor Operação'] = pd.to_numeric(df_entrada['Valor Operação'], errors='coerce')
df_entrada['Valor BC ICMS'] = pd.to_numeric(df_entrada['Valor BC ICMS'], errors='coerce')

df_entrada = df_entrada.astype(schema_entrada)
colunas_novas = {
    'Inscrição Estadual': 'inscricao_estadual',             
    'CPF/CNPJ': 'CPFCNPJ',                                   
    'Ano de referência': 'ano_referencia',                 
    'Mês de apuração': 'mes_apuracao',                      
    'Hash do arquivo transmitido': 'hash_arquivo',          
    'Origem da Informação': 'origem_informacao',           
    'Cód. Município Origem D100': 'cod_municipio_origem_D100',   
    'CFOP': 'CFOP',                                         
    'Valor Operação': 'vl_operacao',                    
    'Valor BC ICMS': 'vl_bc_icms',                      
    'Declaração utilizado?': 'declaracao_utilizado',        
    'Registro utilizado no calculo VA?': 'registro_calculo_va'                         
}

df_entrada = df_entrada.astype(schema_entrada)

df_entrada.to_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\parquet\\efd_base_entradas.parquet')

In [ ]:
df_entrada

In [ ]:
df_entrada = pd.read_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\parquet\\efd_base_entradas.parquet')


In [ ]:
df_entrada.info()

In [ ]:
df_entrada[df_entrada['Cód. Município Origem D100'] == 3301900.0]

In [ ]:
duckdb.sql("""
SELECT 
  SUM(CAST("Valor Operação" AS FLOAT)) AS total_operacao 

FROM df_entrada
where "Cód. Município Origem D100" = 3301900.0
           """).show()

## SAIDAS

In [179]:
pasta_saida = r'C:\Users\Jose Nelio\Documents\GitHub\ipm_2025\data\efd_base\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811'

In [180]:
tabela_completa_saida = []

for arquivo in os.listdir(pasta_saida):
    if arquivo.endswith('.xlsx') and 'Saida' in arquivo:
        caminho_arquivo_saida = os.path.join(pasta_saida,arquivo)
        print(f'Lendo arquivo: {arquivo}')
        df_saida = pd.read_excel(caminho_arquivo_saida,dtype=str, sheet_name='Dados EFD Saida', skiprows=1, )
        df_saida['parte_arquivo'] = arquivo.split('_')[-2]
        tabela_completa_saida.append(df_saida)

df_saida = pd.concat(tabela_completa_saida, ignore_index=True)
# schema_saida = {
#     'Inscrição Estadual': 'string',
#     'CPF/CNPJ': 'string',
#     'Ano de referência': 'string',
#     'Mês de apuração': 'string',
#     'Hash do arquivo transmitido': 'string',
#     'Origem da Informação': 'string',
#     'Cód. Município Origem D100': 'string',
#     'CFOP': 'string',
#     # 'Valor Operação': 'float',
#     # 'Valor BC ICMS': 'float',
#     'Declaração utilizado?': 'string',
#     'Registro utilizado no calculo VA?': 'string'
# }

# df_saida = df_saida.astype(schema_saida)
colunas_novas = {
    'Inscrição Estadual': 'inscricao_estadual',             
    'CPF/CNPJ': 'CPFCNPJ',                                   
    'Ano de referência': 'ano_referencia',                 
    'Mês de apuração': 'mes_apuracao',                      
    'Hash do arquivo transmitido': 'hash_arquivo',          
    'Origem da Informação': 'origem_informacao',           
    'Cód. Município Origem D100': 'cod_municipio_origem_D100',   
    'CFOP': 'CFOP',                                         
    'Valor Operação': 'vl_operacao',                    
    'Valor BC ICMS': 'vl_bc_icms',                      
    'Declaração utilizado?': 'declaracao_utilizado',        
    'Registro utilizado no calculo VA?': 'registro_calculo_va'                         
}

df_saida = df_saida.rename(columns=colunas_novas)
df_saida['vl_operacao'] = (df_saida['vl_operacao'].str.replace(' ', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float))
df_saida['vl_bc_icms'] = (df_saida['vl_bc_icms'].str.replace(' ', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float))
# df_saida['vl_bc_icms'] = df_saida['vl_bc_icms'].str.replace(' ', '', regex=False)
# df_saida['vl_operacao'] = df_saida['vl_operacao'].str.replace(' ', '', regex=False)
# df_saida['vl_operacao'] = df_saida['vl_operacao'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
# df_saida['vl_bc_icms'] = df_saida['vl_bc_icms'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_saida['CPFCNPJ'] = df_saida['CPFCNPJ'].str.zfill(14)

df_saida.to_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\parquet\\efd_base_saidas.parquet')


Lendo arquivo: EFD-Base-Saida_IPM_2027_AnoBase2025_Provisório_parte1_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo arquivo: EFD-Base-Saida_IPM_2027_AnoBase2025_Provisório_parte2_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo arquivo: EFD-Base-Saida_IPM_2027_AnoBase2025_Provisório_parte3_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo arquivo: EFD-Base-Saida_IPM_2027_AnoBase2025_Provisório_parte4_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo arquivo: EFD-Base-Saida_IPM_2027_AnoBase2025_Provisório_parte5_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# df_saida = pd.read_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\parquet\\efd_base_saidas.parquet')
# df_endereco = pd.read_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\parquet\\endereco.parquet')


In [181]:
df_saida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171558 entries, 0 to 1171557
Data columns (total 13 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   inscricao_estadual         1171558 non-null  object 
 1   CPFCNPJ                    1171558 non-null  object 
 2   ano_referencia             1171558 non-null  object 
 3   mes_apuracao               1171558 non-null  object 
 4   hash_arquivo               1171558 non-null  object 
 5   origem_informacao          1171558 non-null  object 
 6   cod_municipio_origem_D100  1171547 non-null  object 
 7   CFOP                       1171558 non-null  object 
 8   vl_operacao                1171558 non-null  float64
 9   vl_bc_icms                 1171558 non-null  float64
 10  declaracao_utilizado       1171558 non-null  object 
 11  registro_calculo_va        1171558 non-null  object 
 12  parte_arquivo              1171558 non-null  object 
dtypes: float64(2

In [183]:
duckdb.sql("""
SELECT 
df_saida.CPFCNPJ,inscricao_estadual,ano_referencia,mes_apuracao,origem_informacao, cod_municipio_origem_D100,CFOP,df_endereco.localidade_atual,
SUM(
CASE 
           
 --A)
  WHEN 
    origem_informacao IN ('C190', 'C590', 'C790', 'D590', 'D696') AND (CFOP LIKE '5%' OR CFOP LIKE '6%' OR CFOP LIKE '7%')
    AND NOT (CFOP IN ('5151','5152','5153','6151','6152','6153','5155','5156','6155','6156','5208','5209','6208','6209','5408','5409','6408','6409','5658','5659','6658','6659'))
    --AND NOT EXISTS (SELECT * FROM df_saida AS c500 WHERE origem_informacao = 'C500' AND COD_MOD = '29' )
  THEN 
    CAST(vl_operacao AS FLOAT)
  WHEN 
    origem_informacao = 'D190' 
  THEN 
    CAST(vl_operacao AS FLOAT)

 --B)
  WHEN 
    CFOP IN ('5151', '5152', '5153', '6151', '6152', '6153', '5155', '5156', '6155', '6156', '5208', '5209', '6208', '6209', '5408', '5409', '6408', '6409', '5658', '5659', '6658', '6659')
  THEN
    CAST(vl_bc_icms AS FLOAT)
           
  --C) 
  WHEN 
    CFOP IN ('5949', '6949', '7949')
  THEN
    CAST(vl_bc_icms AS FLOAT)
END ) AS VL_APURADO,

  --E)
  CASE
WHEN origem_informacao = 'D190' AND VL_APURADO >= 0.0 THEN cod_municipio_origem_D100 
  ELSE df_endereco.localidade_atual
  END AS municipio_vl
          


FROM df_saida
LEFT JOIN df_endereco ON df_saida.CPFCNPJ = df_endereco.CPFCNPJ
           --WHERE "Cod_Municipio_Origem_D100" = 3301900.0

GROUP BY df_saida.CPFCNPJ,inscricao_estadual,ano_referencia,mes_apuracao,origem_informacao,cod_municipio_origem_D100,CFOP,df_endereco.localidade_atual
           
""").show(max_rows=1635)

┌────────────────┬────────────────────┬────────────────┬──────────────┬───────────────────┬───────────────────────────┬─────────┬─────────────────────────────┬──────────────────────┬─────────────────────────────┐
│    CPFCNPJ     │ inscricao_estadual │ ano_referencia │ mes_apuracao │ origem_informacao │ cod_municipio_origem_D100 │  CFOP   │      localidade_atual       │      VL_APURADO      │        municipio_vl         │
│    varchar     │      varchar       │    varchar     │   varchar    │      varchar      │          varchar          │ varchar │           varchar           │        double        │           varchar           │
├────────────────┼────────────────────┼────────────────┼──────────────┼───────────────────┼───────────────────────────┼─────────┼─────────────────────────────┼──────────────────────┼─────────────────────────────┤
│ 09611669006397 │ 15025824           │ 2025           │ 05           │ C190              │ 0                         │ 6152    │ NITERÓI           